# Vertex list generation

Same as other vertex list generator, but now for triangular mesh (simplicial)




In [1]:
import numpy as np
import json

### Toy example

In [2]:
Nx=3
Nz=4
xs, zs  = np.linspace(-2,2,Nx), np.linspace(-3,3,Nz)

In [3]:
X, Z = np.meshgrid(xs, zs)
X.shape
# X, Z

(4, 3)

What order does the below put x,z coordinates into? (man terrible grammar)

In [84]:
data = np.arange(0,Z.shape[0]*Z.shape[1]).reshape(Z.shape)
datac = np.arange(Z.shape[0]*Z.shape[1], 2*Z.shape[0]*Z.shape[1]).reshape(Z.shape)

In [86]:
complexdata = data + 1j*datac
complexdata

array([[ 0.+12.j,  1.+13.j,  2.+14.j],
       [ 3.+15.j,  4.+16.j,  5.+17.j],
       [ 6.+18.j,  7.+19.j,  8.+20.j],
       [ 9.+21.j, 10.+22.j, 11.+23.j]])

In [87]:
data, X, Z

(array([[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8],
        [ 9, 10, 11]]),
 array([[-2.,  0.,  2.],
        [-2.,  0.,  2.],
        [-2.,  0.,  2.],
        [-2.,  0.,  2.]]),
 array([[-3., -3., -3.],
        [-1., -1., -1.],
        [ 1.,  1.,  1.],
        [ 3.,  3.,  3.]]))

In [88]:
I = np.array([X, data, Z])

In [89]:
Verts = I.T.reshape((Nx*Nz,3))
Verts

array([[-2.,  0., -3.],
       [-2.,  3., -1.],
       [-2.,  6.,  1.],
       [-2.,  9.,  3.],
       [ 0.,  1., -3.],
       [ 0.,  4., -1.],
       [ 0.,  7.,  1.],
       [ 0., 10.,  3.],
       [ 2.,  2., -3.],
       [ 2.,  5., -1.],
       [ 2.,  8.,  1.],
       [ 2., 11.,  3.]])

Lexicographic in x coordinate. Track complex data:

In [103]:
Data = np.array([complexdata.real, complexdata.imag]).T.reshape((Nx*Nz,2))
Data

array([[ 0., 12.],
       [ 3., 15.],
       [ 6., 18.],
       [ 9., 21.],
       [ 1., 13.],
       [ 4., 16.],
       [ 7., 19.],
       [10., 22.],
       [ 2., 14.],
       [ 5., 17.],
       [ 8., 20.],
       [11., 23.]])

Yep, that works for the imag data.

Indices below track index of vertices in listed format from 2 cells up to original location in 2d array:

In [96]:
inds = np.arange(0, Nx*Nz).reshape((Nx, Nz)).T

In [97]:
inds  # shape is same as X, Z in meshgrid, but entries give index in later lexicographic ordering.

array([[ 0,  4,  8],
       [ 1,  5,  9],
       [ 2,  6, 10],
       [ 3,  7, 11]])

We'll make face index lists using the upper left coordinates as the bases.  Let's get those:

In [15]:
inds.shape

(4, 3)

In [16]:
bases = inds[:Nz-1, :Nx-1]  # right, that's easy, just a slice  (up to len(zs)-1, len(xs)-1 as we transposed)
fbases = bases.T.flatten()
bases, fbases

(array([[0, 4],
        [1, 5],
        [2, 6]]),
 array([0, 1, 2, 4, 5, 6]))

To make face index data, could loop over this base list.  Looks like the list will be the index at hand, say 0 at first, followed by that index plus len(z), here that gives 4, then that number +1, 5, then the original index plus one.  (Well that's what we did for quads, here we alter that to make trigs):

In [17]:
test_faces = np.array([[i, i+Nz, i+Nz+1, i+1] for i in fbases])
test_trigs = np.concatenate((
    np.array([[i, i+Nz, i+1] for i in fbases]),
    np.array([[i+1, i+Nz, i+Nz+1]for i in fbases])))
test_trigs

array([[ 0,  4,  1],
       [ 1,  5,  2],
       [ 2,  6,  3],
       [ 4,  8,  5],
       [ 5,  9,  6],
       [ 6, 10,  7],
       [ 1,  4,  5],
       [ 2,  5,  6],
       [ 3,  6,  7],
       [ 5,  8,  9],
       [ 6,  9, 10],
       [ 7, 10, 11]])

In [16]:
def trig_faces(xs, zs=None):
    '''Get face index list for blender meshing from domain mesh data.'''
    if len(xs.shape) > 1:  # assume meshgrid data provided.
        Nz, Nx = xs.shape[0], xs.shape[1]  # Dimension swap? Check
        l_inds = Nx * Nz
    elif len(xs.shape) == 1 and zs is not None:
        Nx, Nz= len(xs), len(zs)
        l_inds = Nx * Nz
    else:
        raise ValueError('Must provide either meshgrid (2d array) or two 1d arrays xs and zs as keyword argument zs=.')
    indices =  np.arange(0, l_inds).reshape((Nx, Nz)).T  # Note transpose
    fbases = indices[:Nz-1, :Nx-1].T.flatten()  # Note transpose and index switch due to transpose
    f1 = np.array([[i, i+Nz, i+1] for i in fbases], dtype=int)
    f2 = np.array([[i+1, i+Nz, i+Nz+1] for i in fbases], dtype=int)
    return np.concatenate((f1, f2))
    

In [19]:
test_trigs, trig_faces(X)

(array([[ 0,  4,  1],
        [ 1,  5,  2],
        [ 2,  6,  3],
        [ 4,  8,  5],
        [ 5,  9,  6],
        [ 6, 10,  7],
        [ 1,  4,  5],
        [ 2,  5,  6],
        [ 3,  6,  7],
        [ 5,  8,  9],
        [ 6,  9, 10],
        [ 7, 10, 11]]),
 array([[ 0,  4,  1],
        [ 1,  5,  2],
        [ 2,  6,  3],
        [ 4,  8,  5],
        [ 5,  9,  6],
        [ 6, 10,  7],
        [ 1,  4,  5],
        [ 2,  5,  6],
        [ 3,  6,  7],
        [ 5,  8,  9],
        [ 6,  9, 10],
        [ 7, 10, 11]]))

In [20]:
np.save('/home/pv/test_trig_faces_zero_data', trig_faces(X))
np.save('/home/pv/test_vertices_zero_data', Verts)

In [21]:
with open('test_vertices_zero_data.json', 'w') as outfile:
    
    json.dump(Verts.flatten().tolist(), outfile, 
              separators=(',', ':'), 
              sort_keys=True, 
              indent=4) ### this saves the array in .json format


In [22]:
with open('test_trig_faces_zero_data.json', 'w') as outfile:
    
    json.dump(trig_faces(X).flatten().tolist(), outfile, 
              separators=(',', ':'), 
              sort_keys=True, 
              indent=4) ### this saves the array in .json format


In [25]:
np.save('/home/pv/learning/javascript/test_trig_faces_zero_data', trig_faces(X))
np.save('/home/pv/learning/javascript/test_vertices_zero_data', Verts)

# Actual data

In [9]:
ys = np.load('rad_field.npy')
Xs, Zs = np.load('radX.npy'), np.load('radZ.npy')
L = Xs.shape[0] * Xs.shape[1]


Need to save ys as [ys[0].real, ys[0].imag, ys[1].real, ys[1].imag, ... ]

In [10]:
Ys = np.array([ys.real, ys.imag], dtype=float)

In [11]:
Ys.T.reshape((L,2))

array([[-1.75692326e-06,  0.00000000e+00],
       [-2.43072236e-05,  1.57549405e-05],
       [-4.53495696e-05,  3.15002032e-05],
       ...,
       [-1.13221953e-02, -3.70223793e-04],
       [-1.06400115e-02, -2.24347756e-03],
       [-9.65314658e-03, -3.94561749e-03]])

Note that we checked above that this gives correct ordering. Let's set this as height data variable

In [12]:
heights = Ys.T.reshape((L,2))

In [13]:
Base = np.array([Xs, np.zeros_like(Xs), Zs], dtype=float)


In [14]:
L = Xs.shape[0] * Xs.shape[1]
base_vertices = Base.T.reshape((L,3))


In [17]:
trig_face_indices = trig_faces(Xs)

In [18]:
trig_face_indices.shape

(1148642, 3)

In [19]:
np.max(trig_face_indices)

575999

In [83]:
# np.save('/home/pv/local/blender_test/trig_face_indices', trig_face_indices)
# np.save('/home/pv/local/blender_test/vertices_real', vertices_R)
# np.save('/home/pv/local/blender_test/vertices_imag', vertices_I)

In [123]:
np.save('/home/pv/learning/javascript/trig_face_indices', trig_face_indices)
np.save('/home/pv/learning/javascript/base_vertices', base_vertices)
np.save('/home/pv/learning/javascript/heights', heights)

In [127]:
np.save('/home/pv/learning/index_buffered/trig_face_indices', trig_face_indices)
np.save('/home/pv/learning/index_buffered/base_vertices', base_vertices)
np.save('/home/pv/learning/index_buffered/heights', heights)

In [128]:
prefix = '/home/pv/learning/index_buffered/'

In [126]:
import json

In [129]:

with open(prefix + 'heights.json', 'w') as outfile:
    
    json.dump(heights.flatten().tolist(), outfile, 
              separators=(',', ':'), 
              sort_keys=True, 
              indent=4) ### this saves the array in .json format


In [132]:

with open(prefix + 'base_vertices.json', 'w') as outfile:
    
    json.dump(base_vertices.flatten().tolist(), outfile, 
              separators=(',', ':'), 
              sort_keys=True, 
              indent=4) ### this saves the array in .json format


In [133]:

with open(prefix + 'trig_face_indices.json', 'w') as outfile:
    
    json.dump(trig_face_indices.flatten().tolist(), outfile, 
              separators=(',', ':'), 
              sort_keys=True, 
              indent=4) ### this saves the array in .json format
